In [1]:
import sys
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

In [2]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML table cell
    Input: the element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML table cell
    Input: the element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i, booster_version in enumerate(table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell
    Input: the element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out

def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass

def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    column_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(column_name.strip().isdigit()):
        column_name = column_name.strip()
        return column_name

In [3]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

In [4]:
data = requests.get(static_url).text

In [5]:
soup = BeautifulSoup(data)

In [6]:
html_tables = soup.find_all('table')

In [7]:
#Let's print the third table and check its content
first_launch_table = html_tables[2]
#print(first_launch_table)

In [13]:
column_names = []
rows = []

for i, row in enumerate(first_launch_table.find_all('tr')):
    if i == 0:
        column_names = [el.text.strip() for el in row.find_all('th')]
    else:
        rows.append([el.text.strip() for el in row.find_all('td')])

print(column_names)

['Flight No.', 'Date andtime (UTC)', 'Version,Booster [b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding']


In [14]:
column_names[1] = 'Date and time ( )'
column_names[2] = 'Version Booster'
column_names[4] = 'Payload'
column_names[8] = 'Launch outcome'
column_names[9] = 'Booster landing'

In [99]:
launch_dict = dict.fromkeys(column_names)

# Remove an irrelevant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Version Booster'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
launch_dict['Booster landing'] = []
# Added some new columns
launch_dict['Date'] = []
launch_dict['Time'] = []

In [100]:
print(launch_dict)

{'Flight No.': [], 'Version Booster': [], 'Launch site': [], 'Payload': [], 'Payload mass': [], 'Orbit': [], 'Customer': [], 'Launch outcome': [], 'Booster landing': [], 'Date': [], 'Time': []}


In [101]:
No, Date, Time, BV, LS, PL, PLM, Orbit, Customer, LO, BL = [], [], [], [], [], [], [], [], [], [], []

In [102]:
extracted_row = 0
#Extract each table
for table_number, table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   #get table row
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element
        row=rows.find_all('td')
        #if it is number save cells in a dictonary
        if flag:
            extracted_row += 1
            
            #print(flight_number)
            datatimelist=date_time(row[0])
            No.append(flight_number)
            
            
            date = datatimelist[0].strip(',')
            #print(date)
            Date.append(date)
            
            
            time = datatimelist[1]
            #print(time)
            Time.append(time)
              
            
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            #print(bv)
            BV.append(bv)
            
            
            launch_site = row[2].a.string
            #print(launch_site)
            LS.append(launch_site)
            
            
            payload = row[3].a.string
            #print(payload)
            PL.append(payload)
            
            
            payload_mass = get_mass(row[4])
            #print(payload_mass)
            PLM.append(payload_mass)
            
            
            orbit = row[5].a.string
            #print(orbit)
            Orbit.append(orbit)
            
            
            customer = row[6]
            #print(customer)
            Customer.append(customer.text)
            
            
            launch_outcome = list(row[7].strings)[0]
            #print(launch_outcome)
            LO.append(launch_outcome)
            
            
            booster_landing = landing_status(row[8])
            #print(booster_landing)
            BL.append(booster_landing)

In [108]:
launch_dict = {'No' : No, 
               'Date' : Date, 
               'Time' : Time, 
               'Version_Booster' : BV, 
               'Launch_site' : LS, 
               'Payload' : PL, 
               'Payload_mass' : PLM, 
               'Orbit' : Orbit, 
               'Customer' : Customer, 
               'Launch_outcome' : LO, 
               'Booster_landing' : BL,
              }

In [110]:
df = pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

In [112]:
df.head()

,No,Date,Time,Version_Booster,Launch_site,Payload,Payload_mass,Orbit,Customer,Launch_outcome,Booster_landing
0,1,4 June 2010,18:45,F9 v1.0B0003.1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX\n,Success\n,Failure
1,2,8 December 2010,15:43,F9 v1.0B0004.1,CCAFS,Dragon,0,LEO,\nNASA (COTS)\nNRO\n\n,Success,Failure
2,3,22 May 2012,07:44,F9 v1.0B0005.1,CCAFS,Dragon,525 kg,LEO,NASA (COTS)\n,Success,Not attempted\n
3,4,8 October 2012,00:35,F9 v1.0B0006.1,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA (CRS)\n,Success\n,No attempt
4,5,1 March 2013,15:10,F9 v1.0B0007.1,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA (CRS)\n,Success\n,Not attempted\n


In [ ]:
df.to_csv('spacex_web_scraped.csv', index=False)